Extractor de características

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import hsv_to_rgb
# from matplotlib import cm
from matplotlib import colors
import os
from IPython.display import clear_output
import time 
import math
import statistics

In [ ]:
light_yellow = np.array([0,120,20],np.uint8)
dark_yellow = np.array([175,255,255],np.uint8)

In [ ]:
path = 'fruit_dataset/training/'

In [ ]:
# obtenemos lista de imagenes a analizar
list = os.listdir(path)
print(len(list))

In [ ]:
hue_feature = []
cva_feature = []
print(hue_feature)


In [ ]:
for i in range(len(list)):
    plt.rcParams.update({'figure.max_open_warning': 0})
    img = cv2.imread(path+list[i])
    print('('+str(i)+'): '+path+list[i])
    
    # Reservamos la imagen original
    img_original = img 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    r,g,b = cv2.split(img)
      
    # Plot 1
    fig = plt.figure(figsize=(8,6),dpi=80)
    axis = fig.add_subplot(1,2,1,projection="3d")
    
    # Configuración de color de píxeles
    pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1],3))

    # Normalizado
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    # Convertir a lista
    pixel_colors = norm(pixel_colors).tolist()

    # Mostrar diagrama de dispersión 3D
    axis.scatter(r.flatten(),g.flatten(),b.flatten(),facecolors=pixel_colors,marker='.')
    axis.set_xlabel("Red")
    axis.set_ylabel("Green")
    axis.set_zlabel("Blue")

    # Visualización en espacio hsv
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    h,s,v = cv2.split(hsv_img)
    
    # Plot 2
    fig = plt.figure(figsize=(8,6),dpi=80)
    axis = fig.add_subplot(1,2,2,projection="3d")

    # Configuración de color de píxeles
    pixel_colors = img.reshape((np.shape(img)[0]*np.shape(img)[1],3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    # Convertir a lista
    pixel_colors = norm(pixel_colors).tolist()

    # Mostrar diagrama de dispersión 3D
    axis.scatter(h.flatten(),s.flatten(),v.flatten(),facecolors=pixel_colors,marker='.')
    axis.set_xlabel("hue")
    axis.set_ylabel("saturation")
    axis.set_zlabel("value")
    
    #plt.show()
    
    # Hue feature
    
    h_mean = np.array(h)
    positives = h_mean > 0 # Ignoramos los 0 que corresponden al blanco
    if positives.any():
        h_mean = h_mean[positives].mean()
    
    print('h_mean: ',h_mean)
    
    hue_feature.append(h_mean)
    print(hue_feature)
    
    # Center of gravity and circle variance
    
    mask = cv2.inRange(hsv_img,light_yellow,dark_yellow)
    contours, he = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #cv2.drawContours(img_original, contours, -1, (255,0,0), 3)
    
    for c in contours:
        area = cv2.contourArea(c)
        if area > 5000:
            M = cv2.moments(c)
            if (M["m00"]==0): M["m00"]=1
            # Centriode
            x = int(M["m10"]/M["m00"])
            y = int(M['m01']/M['m00']) 
            centroide = np.array((x,y))
            cv2.circle(img_original, (x,y), 7, (0,255,0), -1)
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(img_original, '{},{}'.format(x,y),(x+10,y), font, 0.75,(0,255,0),1,cv2.LINE_AA)
            
            nuevoContorno = cv2.convexHull(c)
            
            circularity = cv2.drawContours(img_original, [nuevoContorno], 0, (255,0,0), 3)
   
    distances = []
    distances.clear()
    for i in range(0,len(c),2):
        point = np.array((c.item(i),c.item(i+1)))
        
        dis = math.dist(centroide,point)
        #print(point)
        #print(dis)
        distances.append(dis)
    
    # Cuando finalice obtener media y desviación estandar
   
    mean = statistics.mean(distances)
    st_dev = statistics.pstdev(distances)
    
    cva = st_dev/mean
    cva_feature.append(cva_feature)
    
    print('mean: ',mean)
    print('stdev: ',st_dev)
    
    
    
    #cv2.imshow('maskAzul',mask)
    #cv2.imshow('original',img_original)
    #cv2.waitKey()
    
    cv2.destroyAllWindows()

    #clear_output(wait=True)
    